# AA1 lab 04

# Linear Regression: Life Expectancy Prediction

In [ ]:
# Uncomment to upgrade packages
# !pip3 install pandas --user --upgrade --quiet
# !pip3 install numpy --user --upgrade --quiet
# !pip3 install scipy --user --upgrade --quiet
# !pip3 install statsmodels --user --upgrade --quiet
# !pip3 install seaborn --user --upgrade --quiet
# !pip3 install pillow --user --upgrade -- quiet
# !pip3 install matplotlib --user --upgrade --quiet

%load_ext autoreload

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler

from statsmodels.genmod.generalized_linear_model import GLM
import statsmodels.api as sm
from scipy import stats

import warnings

warnings.filterwarnings("ignore")

sns.set()
pd.set_option("display.precision", 3)

## SECTION 1: Our data

In this lab session we are going to predict the life expentancy of different countries during different years given different socio-economic markers.

As you can notice this dataset has a temporal component (life expectancy in a country one year can have a correlation with previous and posterior years). Normally this temporal component should be taken into account (breaks the iid assumption for example), however for simplicity we will ignore it. Treating the temporal component is outside the scope of the course.

In [ ]:
# load data from file
life_expectancy_data = pd.read_csv("Life_Expectancy_Data.csv")
# remove spaces and symbols to avoid problems with statsmodel GLM
life_expectancy_data.columns = [
    c.lower().strip().replace(" ", "_").replace("/", "_").replace("-", "_")
    for c in life_expectancy_data.columns
]

# change the type of categorical variables into category
categorical_columns = list(
    life_expectancy_data.dtypes[life_expectancy_data.dtypes == "O"].index.values
)
for column in categorical_columns:
    life_expectancy_data[column] = life_expectancy_data[column].astype("category")

# peak into the data
life_expectancy_data.head()

It is always a good idea to do a small exploration of the data. Real data needs preprocessing and it is important to understand your dataset to be able to take good design decisions. 

Our dataset has 2938 samples and 21 predictive variables. Our target is `life_expectancy`. 

We are going to do a fast visualizacion of the data. In this visualization we use histograms for showing the distribution of the numerical variables and barplots for the categorical ones. 

With these kinds of easy visualizations we can see a lot of relevant information about our data, like if we have ourliers or if any variable has been wrongly encoded, or whether it "looks" Gaussian enough.

In [ ]:
fig, axes = plt.subplots(6, 4, figsize=(26, 20))

# We will not plot country because it has too many categories.
for i, c in enumerate(life_expectancy_data.columns[1:]):
    ax = axes.reshape(-1)[i]
    if life_expectancy_data[c].dtype.kind == "O":
        a = sns.countplot(x=c, data=life_expectancy_data, ax=ax)
    else:
        b = sns.histplot(x=c, data=life_expectancy_data, ax=ax)
    t = ax.set_title(c)
plt.tight_layout()

If you can't plot a effectively a categorical variable because it has too many categories, you can check it with a value_counts function.

In [ ]:
life_expectancy_data["country"].value_counts()

Now that we know what our data looks like, it is a good idea to check how many missing values do we have on each variable.  

In [ ]:
life_expectancy_data.isna().sum()

We can use all this information to design an effective pre-processing methodology and to choose an appropiate model adapted to our data. To show how important this is we will do two different pre-processings:
* **Generic Preprocessing**: Just transforms the data to something a model can process without throwing errors. Ignoring completely the dataset peculiarities. 
* **Specific Preprocessing**: Preprocessing adapted to the models we will be using and the data particularities.  

### Resampling protocol

We will use two data partitions (train and test) with cross-validation over the train partition for deciding hyperparameters.  We will explain in detail cross-validation on section 3. 

In [ ]:
X = life_expectancy_data.loc[:, life_expectancy_data.columns != "life_expectancy"]
y = life_expectancy_data["life_expectancy"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)

## SECTION 2: Linear Regression (with minimum preprocessing of data)

This is the Linear Regression lab sesion. For this reason we will be using the Linear Regression, Lasso Regression and Rigdge Regression models. These models are not compatible with missing values nor categorical variables. So, the absolute minimum we need to do to our data is to remove samples with missings and categorical variables.

In [ ]:
def minimum_preprocessing(X, y):
    print("Original shape:{}".format(X.shape))
    categorical_columns = X.dtypes[X.dtypes == "category"].index.values
    # We kill categorical columns
    X = X.drop(columns=categorical_columns)
    print("Droped: {}".format(categorical_columns))
    # We remove missing values
    X = X.dropna()
    y = y[X.index]
    print("New shape:{}".format(X.shape))
    return X, y

We will aply separately our preprocessing to our train and test partitions to avoid generating biases on our data. 

In [ ]:
X_train, y_train = minimum_preprocessing(X_train, y_train)
X_test, y_test = minimum_preprocessing(X_test, y_test)

### Linear Regression

Small recap of how linear regression works: 

We model our regression function as 

 $y = f(x) + \epsilon = w^\top x  + \epsilon$
 
 where: 
 * $y$ is our target.
 * $w$ are the weights we will calculate.
 * $x$ are our samples.
 * $\epsilon$ is the independent term. 

If we assume the error of all variables is gaussian solving this problem is equivalent to minimize the mean squared error of this function. 

$\min_w || y - Xw ||^2$

We will see two different implementations of linear regression. The one from statsmodels and the one from sklearn. 


#### Sklearn linear regression

Sklearn models are really easy to use. They are all implemented on classes with the same structure, so knowing one you know all. 

In [ ]:
# We instantiate a linear regression.
lr = LinearRegression()

# Now we train it on train data with fit method
lr.fit(X_train, y_train)

# We can predict using the predict method
y_pred = lr.predict(X_train)

weights = lr.coef_
intercept = lr.intercept_
# You can access to some info about the model, like the weights.
print("Coefficients: \n", weights[:10])
print("Intercept: \n", intercept)


#### Statsmodels linear regression
*Statsmodels linear regression* is a bit more difficult to use but it generates a lot of statistical data that can be useful. 

In [ ]:
# data_train = X_train.copy()
# data_train['life_expectancy'] = y_train
# data_train = sm.add_constant(data_train)
# Linear regression is called ordinary least squares (OLS) in statsmodels
model = sm.OLS(y_train, sm.add_constant(X_train))
result = model.fit()
result.summary()

Interpreting this output:

Each individual sample will be an array with the next structure:

$x=(1, adult\_mortality, infant\_deaths,\dots, scholing)^T$

The weights of the model after training are: 

$w=(53.8131	, -0.017, 0.1059, \dots,0.9182)^T$

And the model: 

$y(x; w) = w^T x = 53.8131 - 0.017*adult\_mortality + 0.1059*infant\_deaths + \dots + 0.9182*scholing$

Apart from the weights, statsmodels returns the 95% confidence interval of these weights ([0.025	0.975]), the standard error of the weights (std err) and the p value (P>|z|). If p value is smaller to a threshold (usually 0.05) we can say that the variable is relevant for predicting the target.

The residuals (the difference between the actual target value and the predicted target value) are:

 $(t_n - y(x_n; w)), n = 1,\dots N$

If we plot the residuals of the training data we obtain the next distribution:

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.set_xlim([-5, 5])
sns.distplot(result.resid, bins=100);

We want this plot to look Gaussian, as it is our departing assumption (Gaussian error). For this reason this plot is a direct indicator of model validity.

Another plot we can use to validate our model is a QQ-plot.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
stats.probplot(result.resid, plot=plt);

## SECTION 3: Metrics

There are alternative metrics we can use to measure the performance of a regression model. 
We will review the most common ones over the training data predictions. 

**Mean Squared Error (MSE)** 

The best possible result would be a MSE of 0 sa it would mean a perfect prediction. 

$MSE(t,y) = \frac{1}{N} \sum_{i=1}^N (t - y(x;w))^2$

In [ ]:
N = X_train.shape[0]

# using statsmodel
prediction = result.predict(sm.add_constant(X_train))
mean_square_error = np.sum((y_train - prediction) ** 2) / N

# You can also use sklearn implementation
mean_square_error_sk = mean_squared_error(y_train, prediction)

mean_square_error, mean_square_error_sk

This number depends on the magnitude of the targets, so we cannot know if it is good or bad as it is. 
It is a very good practise to normalise it by dividing by the unbiased sample variance of the target. This way we obtain the 

**Normalized Mean Squared Error**

Again, the best possible result would be 0, for the same reason.

$norm\_MSE(t,y) = \frac {MSE(t,y)}{\sigma^2(t)} $


In [ ]:
norm_mse = np.sum((y_train - prediction) ** 2) / ((N) * np.var(y_train))

# You can also use sklearn r2 implementation to calculate this value
norm_mse_sk = 1 - r2_score(y_train, prediction)

norm_mse, norm_mse_sk

 If we divide the mean square error by the variance of the targets t,
 we get the proportion of the variability of the target that is NOT explained by the model

 A model with 'norm.mse' equal to 1 is as good as the best constant model
 (namely, the model that always outputs the average of the target)

 models with 'norm.mse' above 0.5 are so so, beyond 0.7 they begin to be quite bad

 models with 'norm.mse' below 0.2 are quite good

**Multiple R-squared (R^2)**

The Multiple R-squared (R^2) (usually used by statisticians) is obtained by subtracting this quantity from one; that is, the proportion of the target variability that is explained by the model; in this case it reaches ~80%

This metric is usually shown between 0 and 1 or in percentage.

In [ ]:
R_squared = 1 - norm_mse

# You can also use sklearn implementation
R_squared_sk = r2_score(y_train, prediction)

R_squared, R_squared_sk

In [ ]:
X_train.shape

We have seen here that the model has good results with the training data, but does not mean our model is a good predictor. 
To have a numerical asessment of its predictive ability we have several options: 
* Get new data (not possible here!)
* Use LOOCV. 
* Predict on an specific validation partition and calculate the relevant measures.
* Use Cross-Validation. 

We haven't saved a specific partition for obtaining validation metrics and we have too many samples (1123 in train) for doing LOOCV fast. For this reason we will do Cross-Validation. 

## SECTION 4: Cross Validation

We need to know how good is our model. We cannot use the train dataset for this because it might give artificially good results. We cannot use the test partition because we will need to compare theses results with the ones obtained on other models. We want to use the test set only at the end, so it can simulate runing the model on new data. This way we can obtain a generalization error as close as possible to the error we would get using the model on completely new data. 

As we cannot use neither the train or the test sets for evaluating the model we will use **Cross-Validation** for computing our metrics, we will use our cross-validation metrics to compare models and take any design decissions.



![](cross_validation.PNG) 

In [ ]:
cross_val_metrics = pd.DataFrame(columns=["MSE", "norm_MSE", "R2"])

kf = KFold(n_splits=5)
i = 1
for train_index, test_index in kf.split(X_train):
    print(
        "Split {}: \n\tTest Folds: [{}] \n\tTrain Folds {}".format(
            i, i, [j for j in range(1, 6) if j != i]
        )
    )

    x_train_fold = X_train.values[train_index]
    y_train_fold = y_train.values[train_index]
    x_test_fold = X_train.values[test_index, :]
    y_test_fold = y_train.values[test_index]

    lr = LinearRegression()
    lr.fit(x_train_fold, y_train_fold)
    y_pred_fold = lr.predict(x_test_fold)
    fold_mse = mean_squared_error(y_test_fold, y_pred_fold)
    fold_nmse = 1 - r2_score(y_test_fold, y_pred_fold)
    fold_r2 = r2_score(y_test_fold, y_pred_fold)
    print("\tMSE: {} NMSE: {} R2: {}".format(fold_mse, fold_nmse, fold_r2))

    cross_val_metrics.loc["Fold {}".format(i), :] = [fold_mse, fold_nmse, fold_r2]
    i += 1


cross_val_metrics.loc["Mean", :] = cross_val_metrics.mean()
cross_val_metrics

We will use the mean of the folds as our metrics. We can also do this using sklearn cros_val_score  method.

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)
folds_r2 = cross_val_score(lr, X_train, y_train, cv=5, scoring="r2")
lr_r2 = np.mean(folds_r2)
folds_r2, lr_r2

## SECTION 5: Regularized Linear Regression: Ridge and Lasso

Now that we have a way to evaluate our model performance, let's see if it improves by adding regularization. 

### Ridge Regression

This time we are going to minimize the following function:

$\min_w (|| y - Xw ||^2 + \lambda * ||w||^2_2)$

The last term penalizes weights being to large (in magnitude). 
The lambda hyperparameter will control how much we are penalizing them. 

Lambda is not computed over the training process like the weights -- it is a *hyper-parameter*. 
So now the question is: How can we find the best lambda for our dataset? 

We have said on the previous section that we can use cross-validation metrics for comparing different models predictive performance. We can do the same to compare the same model with different hyperparameters.

In [ ]:
ridge_cross_val_metrics = pd.DataFrame(columns=["mean MSE", "mean norm_MSE", "mean R2"])
lambdas = [1e-10, 1e-5, 1e-4, 1e-3, 1e-2, 0.1, 0.5, 1, 5, 10, 50, 100]
# We calculate the cross-validation metrics for each lambda
for lambda_val in lambdas:
    kf = KFold(n_splits=5)
    i = 1
    cv_mse = []
    cv_nmse = []
    cv_r2 = []
    # We compute the metrics for each fold and then perform the mean.
    for train_index, test_index in kf.split(X_train):
        x_train_fold = X_train.values[train_index]
        y_train_fold = y_train.values[train_index]
        x_test_fold = X_train.values[test_index, :]
        y_test_fold = y_train.values[test_index]

        lr = Ridge(alpha=lambda_val)
        lr.fit(x_train_fold, y_train_fold)
        y_pred_fold = lr.predict(x_test_fold)
        fold_mse = mean_squared_error(y_test_fold, y_pred_fold)
        fold_nmse = 1 - r2_score(y_test_fold, y_pred_fold)
        fold_r2 = r2_score(y_test_fold, y_pred_fold)
        cv_mse.append(fold_mse)
        cv_nmse.append(fold_nmse)
        cv_r2.append(fold_r2)
    ridge_cross_val_metrics.loc["Lambda={}".format(lambda_val), :] = [
        np.mean(cv_mse),
        np.mean(cv_nmse),
        np.mean(cv_r2),
    ]

ridge_cross_val_metrics.sort_values(by="mean R2", ascending=False)

Another way to do this is ussing sklearn implementation.

In [ ]:
ridge_cv = RidgeCV(alphas=lambdas, cv=5)
ridge_cv.fit(X_train, y_train)

print("Best lambda:", ridge_cv.alpha_, "R2 score:", ridge_cv.best_score_)

Ridge Regression does not seem to improve the results of linear regression. Let's check if Lasso turns out  to work better. 

### Lasso Regression

This time we penalize weights using their L1 norm. 


$\min_w (|| y - Xw ||^2 + \lambda * |w|)$

We will use scikit-learn's CV method to compute the best lambda directly. 

In [ ]:
lasso_cv = LassoCV(alphas=lambdas, cv=5)
lasso_cv.fit(X_train, y_train)

lasso_r2 = np.mean(cross_val_score(lasso_cv, X_train, y_train))

print("Best lambda:", lasso_cv.alpha_, "R2 score:", lasso_r2)

In [ ]:
r2_results = pd.DataFrame(
    {"lr": lr_r2, "ridge_cv": ridge_cv.best_score_, "lasso_cv": lasso_r2},
    index=["CV R2"],
)

r2_results.loc["Train R2", :] = [
    r2_score(y_train, lr.predict(X_train)),
    r2_score(y_train, ridge_cv.predict(X_train)),
    r2_score(y_train, lasso_cv.predict(X_train)),
]
r2_results.loc["lambda", "lr"] = 0
r2_results.loc["lambda", "ridge_cv"] = ridge_cv.alpha_
r2_results.loc["lambda", "lasso_cv"] = lasso_cv.alpha_
r2_results

## SECTION 6: Understanding our models is important

We can't see a great improvement with Lasso either. 

So, why is regularization not working when it is supposed to give better models? 

Let's check the weights of our three models. 

In [ ]:
weights = pd.DataFrame(
    {"lr": lr.coef_, "ridge_cv": ridge_cv.coef_, "lasso_cv": lasso_cv.coef_},
    index=X_train.columns,
)
weights.T

Visually:

In [ ]:
fig, ax = plt.subplots(figsize=(20, 1))
sns.heatmap(
    weights.T.loc[["lr"], :].abs(),
    annot=True,
    linewidths=0.5,
    ax=ax,
    cbar=False,
    xticklabels=False,
)

plt.figure(figsize=(20, 1))
sns.heatmap(
    weights.T.loc[["ridge_cv"], :].abs(),
    annot=True,
    linewidths=0.5,
    cbar=False,
    xticklabels=False,
)

plt.figure(figsize=(20, 1))
sns.heatmap(
    weights.T.loc[["lasso_cv"], :].abs(),
    annot=True,
    linewidths=0.5,
    cbar=False,
    xticklabels=True,
)

We can see some strange patterns in our weights, in the three models we can see a wide range of values. Both lasso and ridge have almost the same weights. 

If we compare the weights with the mean values of our variables we can see that the linear regression is trying to balance the variable ranges. That's probably why our regularized models are not working so well. 

In [ ]:
plt.figure(figsize=(20, 1))
sns.heatmap(
    X_train.mean().to_frame().T.rename(index={0: "Mean"}),
    annot=True,
    linewidths=0.5,
    cbar=False,
    xticklabels=True,
)

We are going to fix this by scaling our data. This way, all variables will have the same range and we will take better advantage of our models.

**Note that the pre-processing is slightly different for train data than for test data. Eseentially, this is to avoid any type of "leakage"**

In [ ]:
def scaling_preprocessing(X, y, scaler=None):
    print("Original shape:{}".format(X.shape))
    categorical_columns = X.dtypes[X.dtypes == "category"].index.values

    # We scale the numerical columns
    numerical_columns = [c for c in X.columns if c not in categorical_columns]
    if scaler is None:
        # We only want the scaler to fit the train data
        scaler = MinMaxScaler()
        X[numerical_columns] = scaler.fit_transform(X[numerical_columns])
    else:
        X[numerical_columns] = scaler.transform(X[numerical_columns])

    # We kill categorical columns
    X = X.drop(columns=categorical_columns)
    print("Droped: {}".format(categorical_columns))
    # We remove missing values
    X = X.dropna()
    y = y[X.index]
    print("New shape:{}".format(X.shape))
    return X, y, scaler


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)
X_train, y_train, scaler = scaling_preprocessing(X_train, y_train)
X_test, y_test, _ = scaling_preprocessing(X_test, y_test, scaler)

Now our variables have more reasonable ranges. Lets see how this affects our model weights and performance.

In [ ]:
plt.figure(figsize=(20, 1))
sns.heatmap(
    X_train.mean().to_frame().T.rename(index={0: "Mean"}),
    annot=True,
    linewidths=0.5,
    cbar=False,
    xticklabels=True,
)

In [ ]:
lr_scaled = LinearRegression()
lr_scaled.fit(X_train, y_train)
r2_lr_scaled = np.mean(cross_val_score(lr_scaled, X_train, y_train, cv=5, scoring="r2"))

ridge_cv_scaled = RidgeCV(alphas=lambdas, cv=5)
ridge_cv_scaled.fit(X_train, y_train)
r2_ridge_scaled = np.mean(
    cross_val_score(ridge_cv_scaled, X_train, y_train, cv=5, scoring="r2")
)


lasso_cv_scaled = LassoCV(alphas=lambdas, cv=5)
lasso_cv_scaled.fit(X_train, y_train)
r2_lasso_scaled = np.mean(
    cross_val_score(ridge_cv_scaled, X_train, y_train, cv=5, scoring="r2")
)

weights = pd.DataFrame(
    {
        "lr scaled": lr_scaled.coef_,
        "ridge_cv scaled": ridge_cv_scaled.coef_,
        "lasso_cv scaled": lasso_cv_scaled.coef_,
    },
    index=X_train.columns,
)
weights.T

In [ ]:
r2_results = pd.DataFrame(
    {
        "lr": r2_lr_scaled,
        "ridge_cv": r2_ridge_scaled,
        "lasso_cv": r2_lasso_scaled,
    },
    index=["CV R2"],
)

r2_results.loc["Train R2", :] = [
    r2_score(y_train, lr_scaled.predict(X_train)),
    r2_score(y_train, ridge_cv_scaled.predict(X_train)),
    r2_score(y_train, lasso_cv_scaled.predict(X_train)),
]
r2_results.loc["lambda", "lr"] = 0
r2_results.loc["lambda", "ridge_cv"] = ridge_cv_scaled.alpha_
r2_results.loc["lambda", "lasso_cv"] = lasso_cv_scaled.alpha_
r2_results

The results haven't improved much. Perhaps including categorical variables helps. 

We have two categorical variables. Country and status. 

In [ ]:
categorical_columns = X.dtypes[X.dtypes == "O"].index.values
categorical_columns

In [ ]:
def categorical_preprocessing(X, y, scaler=None):
    print("Original shape:{}".format(X.shape))
    categorical_columns = X.dtypes[X.dtypes == "category"].index.values
    numerical_columns = [c for c in X.columns if c not in categorical_columns]

    # Scale numerical variables
    if scaler is None:
        # We only want the scaler to fit the train data
        scaler = MinMaxScaler()
        X[numerical_columns] = scaler.fit_transform(X[numerical_columns])
    else:
        X[numerical_columns] = scaler.transform(X[numerical_columns])

    # Apply one hot encoding to categorical variables
    for column in categorical_columns:
        X_one_hot = pd.get_dummies(X[column], prefix=column)
        X = X.merge(X_one_hot, left_index=True, right_index=True)
        X = X.drop(columns=[column])

    # Drop missings
    X = X.dropna()
    y = y[X.index]
    print("New shape:{}".format(X.shape))
    return X, y, scaler


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)
X_train, y_train, scaler = categorical_preprocessing(X_train, y_train)
X_test, y_test, _ = categorical_preprocessing(X_test, y_test, scaler)

If we use this new pre-processing we obtain super good results on the training set but super bad results on cross-validation. 

This means our model wont be able to generalize if we try to use it on new data. 

How can it be that having more information our model is performing worse?

In [ ]:
lr_one_hot = LinearRegression()
lr_one_hot.fit(X_train, y_train)

r2_lr_one_hot_train = lr_one_hot.score(X_train, y_train)
r2_lr_one_hot_cv = np.mean(
    cross_val_score(lr_one_hot, X_train, y_train, cv=5, scoring="r2")
)

print(
    "Train R2 score: {}\nCross-Validation R2 score: {}".format(
        r2_lr_one_hot_train, r2_lr_one_hot_cv
    )
)

We have two problems with our model: 
1. It is **overfitting** the training data. So, when it receives new data that has not been used on training it is not able to predict. 
2. This might be caused by the **curse of dimensionallity**. We have 214 variables for 1123 samples, which is not a great ratio. 

We can also see on the weights of our model that there is something wrong, as there are differences between the weight ranges. 

In [ ]:
plt.figure(figsize=(20, 1))
sns.heatmap(pd.DataFrame({"lr overfitted": lr_one_hot.coef_}).T)

Finally, if we check what our data looks like, we will see that we have a very sparse matrix. So, on top of all our problems we are trying to use SVD on a sparse matrix, which is not ideal. 

In [ ]:
plt.figure(figsize=(15, 5))
sns.heatmap(X_test.to_numpy(dtype=float));

For fixing overfitting we can either remove this variable or apply regularization. Let's see how our regularized models behave.

In [ ]:
ridge_cv_one_hot = RidgeCV(alphas=lambdas, cv=5)
ridge_cv_one_hot.fit(X_train, y_train)

r2_ridge_one_hot_train = ridge_cv_one_hot.score(X_train, y_train)
r2_ridge_one_hot_cv = np.mean(
    cross_val_score(ridge_cv_one_hot, X_train, y_train, cv=5, scoring="r2")
)

lasso_cv_one_hot = LassoCV(alphas=lambdas, cv=5)
lasso_cv_one_hot.fit(X_train, y_train)

r2_lasso_one_hot_train = ridge_cv_one_hot.score(X_train, y_train)
r2_lasso_one_hot_cv = np.mean(
    cross_val_score(lasso_cv_one_hot, X_train, y_train, cv=5, scoring="r2")
)

weights = pd.DataFrame(
    {
        "lr_one_hot": lr_one_hot.coef_,
        "ridge_cv_one_hot": ridge_cv_one_hot.coef_,
        "lasso_cv_one_hot": lasso_cv_one_hot.coef_,
    },
    index=X_train.columns,
)

for column in weights.columns:
    fig = plt.figure(figsize=(20, 1))
    ax = sns.heatmap(weights[[column]].T)

We obtain smaller weights on our regularized models. Will this affect our models performance?

In [ ]:
r2_results.loc[:, "lr_one_hot"] = [r2_lr_one_hot_cv, r2_lr_one_hot_train, 0]
r2_results.loc[:, "ridge_cv_one_hot"] = [
    r2_ridge_one_hot_cv,
    r2_ridge_one_hot_train,
    ridge_cv_one_hot.alpha_,
]
r2_results.loc[:, "lasso_cv_one_hot"] = [
    r2_lasso_one_hot_cv,
    r2_lasso_one_hot_train,
    lasso_cv_one_hot.alpha_,
]
r2_results

We can see in our cv metrics that regularization improves performance **a lot**. This way the model can take advantage of the extra data we are feeding it and effectively control overfitting. 

## SECTION 7: Model selection

Now we use the cv R2 to choose the best model and give the test R2 as our resulting R2 measure. According to our validation metrics, the best model is ridge_cv with a lambda of 0.001 used with the standarized dataset with categorical variables. 

Now that we have taken all the decisions concerning our model and data pre-processing. We use the test set to see how the model generalizes. This step is supposed to simulate seen how the model performs with *completely new* data, namely, data it has never seen before.

**Typically, before testing the "winning method" we would re-fit it on the whole training data; the RidgeCV method does this by default for us so we do not need to do this here.**

It is important to use the test set only to give the final performance, otherwise we risk giving an over optimistic result. 

In [ ]:
y_tes_predicted = ridge_cv_one_hot.predict(X_test)
r2_ridge = ridge_cv_one_hot.score(X_test, y_test)

print(
    "Mean sqared error with test data: {}".format(
        mean_squared_error(y_test, y_tes_predicted)
    )
)
print("R2 score with test data: {}".format(r2_ridge))